## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_style('darkgrid')
sns.set_palette('magma')

## Importing Data

In [2]:
# California State
ca = pd.read_csv('Datasets/alt_fuel_stations_CA.csv', encoding='latin-1')
la = pd.read_csv('Datasets/City_Owned_Parking_Lots_LA.csv')
sf = pd.read_csv('Datasets/SFMTA_Managed_Off-street_Parking_SF.csv')
sf2 = pd.read_csv('Datasets/greentrip-parking-database_SF.csv')

# Washington State
wa = pd.read_csv('Datasets/alt_fuel_stations_WA.csv', encoding='latin-1')
seattle = pd.read_csv('Datasets/Public_Garages_or_Parking_Lots_Seattle.csv')

In [3]:
ca.head()

,ï»¿Fuel Type Code,Station Name,Street Address,City,State,ZIP,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,Latitude,Longitude,EV Connector Types,Facility Type,EV Pricing
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,NaN,12.0,NaN,34.040539,-118.271387,J1772,PARKING_GARAGE,NaN
1,ELEC,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,NaN,3.0,NaN,34.068720,-118.064000,J1772,STATE_GOV,Free
2,ELEC,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,NaN,1.0,NaN,32.899470,-117.243000,J1772,HOSPITAL,Pay to Park
3,ELEC,San Diego Wild Animal Park,15500 San Pasqual Valley Rd,Escondido,CA,92027,NaN,2.0,NaN,33.098589,-117.004433,J1772,PAY_LOT,Free
4,ELEC,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,NaN,2.0,NaN,34.221665,-118.468371,J1772,CAR_DEALER,Free


In [4]:
ca['City'].value_counts()

Los Angeles        1271
San Diego           576
Irvine              539
San Jose            523
San Francisco       452
                   ... 
Harbor City           1
Waterford             1
Monte Sereno          1
American Canyon       1
Julian                1
Name: City, Length: 687, dtype: int64

## Function to clean EV Parking Datasets

Firstly, the statewide datasets will need to be cleaned and prepared for further EDA and modeling. As the main purpose of the study is to cover EV charging, the fuel type will all be under `ELEC`, therefore the `ï»¿Fuel Type Code` can be dropped as it will not contribute to the predictive process.

Furthermore, the `NaN` values within the 

In [5]:
def clean(data):
    data = data.drop('ï»¿Fuel Type Code', axis = 1)
    data["EV Level1 EVSE Num"].fillna(0, inplace = True)
    data['EV Level2 EVSE Num'].fillna(0, inplace = True)
    data['EV DC Fast Count'].fillna(0, inplace = True)
    data.columns = data.columns.str.lower()
    data.columns = data.columns.str.replace(' ', '_')
    data['ev_slow'] = data['ev_level1_evse_num'] + data['ev_level2_evse_num']
    
    return (data)

In [6]:
ca = clean(ca)
wa = clean(wa)

In [7]:
ca.head()

,station_name,street_address,city,state,zip,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_count,latitude,longitude,ev_connector_types,facility_type,ev_pricing,ev_slow
0,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,0.0,12.0,0.0,34.040539,-118.271387,J1772,PARKING_GARAGE,NaN,12.0
1,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,0.0,3.0,0.0,34.068720,-118.064000,J1772,STATE_GOV,Free,3.0
2,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,0.0,1.0,0.0,32.899470,-117.243000,J1772,HOSPITAL,Pay to Park,1.0
3,San Diego Wild Animal Park,15500 San Pasqual Valley Rd,Escondido,CA,92027,0.0,2.0,0.0,33.098589,-117.004433,J1772,PAY_LOT,Free,2.0
4,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,0.0,2.0,0.0,34.221665,-118.468371,J1772,CAR_DEALER,Free,2.0


In [8]:
wa.head()

,station_name,street_address,city,state,zip,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_count,latitude,longitude,ev_connector_types,facility_type,ev_pricing,ev_slow
0,City of Lacey - City Hall Parking,420 College St,Lacey,WA,98503,0.0,4.0,0.0,47.044011,-122.822404,J1772,MUNI_GOV,NaN,4.0
1,Avista Corp,1411 E Mission Ave,Spokane,WA,99252,1.0,2.0,0.0,47.673347,-117.388933,J1772 NEMA520,OFFICE_BLDG,Free,3.0
2,Steam Plant Grill,159 S Lincoln St,Spokane,WA,99201,0.0,1.0,0.0,47.654998,-117.425025,J1772,RESTAURANT,Free,1.0
3,BELLEVUE BELLEVUE CH 1,450 110th Ave NE,Bellevue,WA,98004,0.0,2.0,0.0,47.614744,-122.193162,J1772,NaN,NaN,2.0
4,The Evergreen State College,2700 Evergreen Pkwy NW,Olympia,WA,98505,2.0,2.0,0.0,47.071917,-122.976675,J1772 NEMA520,COLLEGE_CAMPUS,NaN,4.0


In [9]:
zip1 = ca.groupby(['zip','facility_type'])['ev_slow', 'ev_dc_fast_count'].sum().reset_index()

<ipython-input-9-5f70193c62c4>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  zip1 = ca.groupby(['zip','facility_type'])['ev_slow', 'ev_dc_fast_count'].sum().reset_index()


In [10]:
zip1

,zip,facility_type,ev_slow,ev_dc_fast_count
0,90001,STORAGE,1.0,0.0
1,90002,STREET_PARKING,1.0,0.0
2,90003,UTILITY,2.0,0.0
3,90006,UTILITY,1.0,0.0
4,90007,COLLEGE_CAMPUS,0.0,1.0
...,...,...,...,...
1475,96150,LIBRARY,2.0,0.0
1476,96150,MUNI_GOV,4.0,0.0
1477,96161,BREWERY_DISTILLERY_WINERY,2.0,0.0
1478,96161,HOTEL,12.0,0.0


In [11]:
la.head()
la.columns = la.columns.str.lower()
la = la.rename(columns={'address': 'street_address', 'lotname': 'station_name'})
la['zip'] = la['zipcode'].str.extract(r'(\d{5})')
la.head()

,x,y,id,facilityid,station_name,community,street_address,city,state,zipcode,...,convenientto,type,hours,hourlycost,dailycost,monthlycost,specialfeatures,spaces,status,zip
0,-118.447320,34.185750,250,601,Friar & Sylmar Parking Garage,Van Nuys,14401 Friar St,Van Nuys,CA,91401-2125,...,San Fernando Valley Civic Center & shops,Operated,5:30am-7pm Mon-Fri,$1.80,$7.20,$72.00,Closed nightly & Sat-Sun all day; max height c...,237,Operational,91401
1,-118.377836,34.167968,251,602,Chandler & Tujunga Parking Lot,North Hollywood,11320 Chandler Blvd,North Hollywood,CA,91601-3130,...,"Post Office, Metro Red Line & Orange Line Stat...",Metered,7am-9pm Daily,$0.50,$2.50,NaN,NaN,46,Operational,91601
2,-118.449273,34.185842,252,609,Friar & Van Nuys Parking Lot,Van Nuys,14521 Friar St,Van Nuys,CA,91401-2309,...,San Fernando Valley Civic Center & shops,Operated,8am-5pm Mon-Sat,$1.10,$5.00,$49.50,$2.00 flat rate after 2pm,76,Operational,91401
3,-118.449489,34.187505,253,610,Gilmore & Van Nuys Parking Lot,Van Nuys,14532 Gilmore St,Van Nuys,CA,91411-1602,...,San Fernando Valley Civic Center & shops,Operated,8am-5pm Mon-Fri,$1.10,$5.00,$38.50,$2.00 flat rate after 2pm,138,Operational,91411
4,-118.458649,33.991500,254,613,Pisani & Venice Parking Lot,Venice,2010 S Pisani Pl,Venice,CA,90291-3817,...,"Pacific Residents Theater, art center & shops",Free,8am-6pm Mon-Sat,Free,Free,NaN,10 hrs max time limit,53,Operational,90291


In [12]:
la['zip'] = la['zip'].astype(str)
la['spaces'] = la['spaces'].str.replace(',', '_')
la['spaces'] = la['spaces'].astype(float)
la.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   x                124 non-null    float64
 1   y                124 non-null    float64
 2   id               124 non-null    int64  
 3   facilityid       124 non-null    object 
 4   station_name     124 non-null    object 
 5   community        119 non-null    object 
 6   street_address   124 non-null    object 
 7   city             124 non-null    object 
 8   state            124 non-null    object 
 9   zipcode          124 non-null    object 
 10  lat              124 non-null    float64
 11  lon              124 non-null    float64
 12  convenientto     117 non-null    object 
 13  type             118 non-null    object 
 14  hours            115 non-null    object 
 15  hourlycost       115 non-null    object 
 16  dailycost        115 non-null    object 
 17  monthlycost     

In [13]:
zip2 = la.groupby(['zip', 'convenientto'])['spaces'].sum().reset_index()

In [14]:
zip2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           67 non-null     object 
 1   convenientto  67 non-null     object 
 2   spaces        67 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.7+ KB


In [15]:
sf2.head()
sf2.columns = sf2.columns.str.lower()
sf2.columns = sf2.columns.str.replace(' ', '_')
sf2.head()

,building_id,building_names,street_address,city,state,zip_code,place_type,square_footage,commercial_square_footage,residential_square_footage,mixed_use,parking_spaces,parking_spaces_off_street
0,101,801 Alma,801 Alma Street,Palo Alto,CA,94301,Local Neighborhood,63885,16082,47803,No,60,60
1,103,Cottonwood Place,3701 Peralta Blvd,Fremont,CA,94536,Transit Neighborhood,101956,40822,61134,Yes,110,110
2,104,Delmas Park Apartments LP,350 Bird Ave,San Jose,CA,95126,Regional Center,-,0,-,Yes,144,144
3,105,EC Magnolia Court,22880 Watkins Street,Hayward,CA,94541,City Center,-,0,16940,No,9,9
4,106,Elena Gardens Apartments,1900 Lakewood Dr.,San Jose,CA,95132,Mixed-Use Corridor,300999,161247,139752,No,220,220


In [16]:
sf2 = sf2.rename(columns={'zip_code': 'zip', 'parking_spaces':'spaces'})
sf2['zip'] = sf2['zip'].astype(str)

In [17]:
zip3 = sf2.groupby(['zip', 'place_type'])['spaces'].sum().reset_index()

In [18]:
zip3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   zip         56 non-null     object
 1   place_type  56 non-null     object
 2   spaces      56 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.4+ KB


In [19]:
parking_1 = pd.merge(zip1, zip2, on = 'zip', how = 'outer')

In [20]:
parking_ca = pd.merge(parking_1, zip3, on ='zip', how = 'outer')

In [21]:
parking_ca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1588 entries, 0 to 1587
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   zip               1588 non-null   object 
 1   facility_type     1565 non-null   object 
 2   ev_slow           1565 non-null   float64
 3   ev_dc_fast_count  1565 non-null   float64
 4   convenientto      130 non-null    object 
 5   spaces_x          130 non-null    float64
 6   place_type        87 non-null     object 
 7   spaces_y          87 non-null     float64
dtypes: float64(4), object(4)
memory usage: 111.7+ KB


In [22]:
parking_ca['spaces_x'] = parking_ca['spaces_x'].fillna(parking_ca['spaces_y'])

In [23]:
parking_ca

,zip,facility_type,ev_slow,ev_dc_fast_count,convenientto,spaces_x,place_type,spaces_y
0,90001,STORAGE,1.0,0.0,NaN,NaN,NaN,NaN
1,90002,STREET_PARKING,1.0,0.0,NaN,NaN,NaN,NaN
2,90003,UTILITY,2.0,0.0,Post Office & shops,61.0,NaN,NaN
3,90003,UTILITY,2.0,0.0,Shops,21.0,NaN,NaN
4,90003,UTILITY,2.0,0.0,Swap meet & shops,64.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1583,95125,NaN,NaN,NaN,NaN,119.0,Local Neighborhood,119.0
1584,95125,NaN,NaN,NaN,NaN,83.0,Transit Town Center,83.0
1585,95130,NaN,NaN,NaN,NaN,50.0,Suburban Center,50.0
1586,95132,NaN,NaN,NaN,NaN,220.0,Mixed-Use Corridor,220.0


In [24]:
parking_ca['facility_type'] = parking_ca['facility_type'].fillna(parking_ca['place_type'])

In [25]:
parking_ca

,zip,facility_type,ev_slow,ev_dc_fast_count,convenientto,spaces_x,place_type,spaces_y
0,90001,STORAGE,1.0,0.0,NaN,NaN,NaN,NaN
1,90002,STREET_PARKING,1.0,0.0,NaN,NaN,NaN,NaN
2,90003,UTILITY,2.0,0.0,Post Office & shops,61.0,NaN,NaN
3,90003,UTILITY,2.0,0.0,Shops,21.0,NaN,NaN
4,90003,UTILITY,2.0,0.0,Swap meet & shops,64.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1583,95125,Local Neighborhood,NaN,NaN,NaN,119.0,Local Neighborhood,119.0
1584,95125,Transit Town Center,NaN,NaN,NaN,83.0,Transit Town Center,83.0
1585,95130,Suburban Center,NaN,NaN,NaN,50.0,Suburban Center,50.0
1586,95132,Mixed-Use Corridor,NaN,NaN,NaN,220.0,Mixed-Use Corridor,220.0


In [26]:
parking_ca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1588 entries, 0 to 1587
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   zip               1588 non-null   object 
 1   facility_type     1581 non-null   object 
 2   ev_slow           1565 non-null   float64
 3   ev_dc_fast_count  1565 non-null   float64
 4   convenientto      130 non-null    object 
 5   spaces_x          217 non-null    float64
 6   place_type        87 non-null     object 
 7   spaces_y          87 non-null     float64
dtypes: float64(4), object(4)
memory usage: 111.7+ KB


In [27]:
wa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   station_name        1511 non-null   object 
 1   street_address      1511 non-null   object 
 2   city                1511 non-null   object 
 3   state               1511 non-null   object 
 4   zip                 1511 non-null   int64  
 5   ev_level1_evse_num  1511 non-null   float64
 6   ev_level2_evse_num  1511 non-null   float64
 7   ev_dc_fast_count    1511 non-null   float64
 8   latitude            1511 non-null   float64
 9   longitude           1511 non-null   float64
 10  ev_connector_types  1511 non-null   object 
 11  facility_type       343 non-null    object 
 12  ev_pricing          598 non-null    object 
 13  ev_slow             1511 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 165.4+ KB


In [28]:
wa['zip'] = wa['zip'].astype(str)

In [29]:
zip4 = wa.groupby(['zip','facility_type'])['ev_slow', 'ev_dc_fast_count'].sum().reset_index()

<ipython-input-29-209fd24fc0d9>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  zip4 = wa.groupby(['zip','facility_type'])['ev_slow', 'ev_dc_fast_count'].sum().reset_index()


In [30]:
seattle.head()

,X,Y,OBJECTID,BUSLIC_LOCATION_ID,DEA_FACILITY_OWNER,DEA_FACILITY_NAME,DEA_BUSINESS_CONTACT,DEA_BUSINESS_PHONE,DEA_FACILITY_ADDRESS,DEA_STALLS,...,DISABLED,OP_NAME,OP_PHONE,OP_PHONE2,OP_WEB,PAYMENT_TYPE,OTHER,WEBNAME,REGIONID,SIGNID
0,-122.318463,47.660851,12873,705410,NaN,DIAMOND PARKING WX10,JONATHAN DIAMOND,2062843100,"907 NE 45TH ST, SEATTLE, WA 98105",17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
1,-122.334638,47.605821,12874,580480,NaN,EXPEDITOR GARAGE,R GATES,2066743400,"1015 3RD AVE # FL 12, SEATTLE, WA 98104",154.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
2,-122.358377,47.620764,12875,625108,NaN,DIAMOND PARKING A128,JONATHAN DIAMOND,2062843100,"233 1ST AVE W, SEATTLE, WA 98109",22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,-122.339572,47.610666,12876,740982,NaN,UNITED PARKING SERVICES 4553,JONATHAN DIAMOND,2062843100,"1600 2ND AVE, SEATTLE, WA 98101-3273",50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,-122.321252,47.610651,12877,711581,NaN,ALCOA LOT 1401,MARK NORWICZ,4102676111,"1401 MADISON ST, SEATTLE, WA 98104-1316",53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [31]:
seattle.columns = seattle.columns.str.lower()

In [32]:
seattle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   x                     700 non-null    float64
 1   y                     700 non-null    float64
 2   objectid              700 non-null    int64  
 3   buslic_location_id    700 non-null    int64  
 4   dea_facility_owner    14 non-null     object 
 5   dea_facility_name     698 non-null    object 
 6   dea_business_contact  656 non-null    object 
 7   dea_business_phone    697 non-null    object 
 8   dea_facility_address  682 non-null    object 
 9   dea_stalls            695 non-null    float64
 10  fac_name              692 non-null    object 
 11  hrs_monfri            26 non-null     object 
 12  hrs_sat               26 non-null     object 
 13  hrs_sun               26 non-null     object 
 14  fac_type              28 non-null     object 
 15  rte_1hr               2

In [33]:
seattle['zip'] = seattle['dea_facility_address'].str.extract(r'(\d{5})')
seattle['zip'] = seattle['zip'].astype(str)

In [34]:
seattle.head()

,x,y,objectid,buslic_location_id,dea_facility_owner,dea_facility_name,dea_business_contact,dea_business_phone,dea_facility_address,dea_stalls,...,op_name,op_phone,op_phone2,op_web,payment_type,other,webname,regionid,signid,zip
0,-122.318463,47.660851,12873,705410,NaN,DIAMOND PARKING WX10,JONATHAN DIAMOND,2062843100,"907 NE 45TH ST, SEATTLE, WA 98105",17.0,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,98105
1,-122.334638,47.605821,12874,580480,NaN,EXPEDITOR GARAGE,R GATES,2066743400,"1015 3RD AVE # FL 12, SEATTLE, WA 98104",154.0,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,98104
2,-122.358377,47.620764,12875,625108,NaN,DIAMOND PARKING A128,JONATHAN DIAMOND,2062843100,"233 1ST AVE W, SEATTLE, WA 98109",22.0,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,98109
3,-122.339572,47.610666,12876,740982,NaN,UNITED PARKING SERVICES 4553,JONATHAN DIAMOND,2062843100,"1600 2ND AVE, SEATTLE, WA 98101-3273",50.0,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,98101
4,-122.321252,47.610651,12877,711581,NaN,ALCOA LOT 1401,MARK NORWICZ,4102676111,"1401 MADISON ST, SEATTLE, WA 98104-1316",53.0,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,98104


In [35]:
seattle = seattle.rename(columns={'dea_stalls':'spaces'})

In [36]:
zip5 = seattle.groupby('zip')['spaces'].sum().reset_index()

In [37]:
parking_wa = pd.merge(zip4, zip5, on ='zip', how = 'outer')

In [38]:
parking_wa.head()

,zip,facility_type,ev_slow,ev_dc_fast_count,spaces
0,98001,OTHER_ENTERTAINMENT,2.0,0.0,NaN
1,98001,SHOPPING_MALL,4.0,14.0,NaN
2,98002,GROCERY,2.0,0.0,NaN
3,98002,SHOPPING_CENTER,4.0,0.0,NaN
4,98003,HOTEL,3.0,0.0,NaN


In [39]:
parking_ca.columns

Index(['zip', 'facility_type', 'ev_slow', 'ev_dc_fast_count', 'convenientto',
       'spaces_x', 'place_type', 'spaces_y'],
      dtype='object')

In [40]:
parking_ca.drop(columns =['convenientto', 'place_type', 'spaces_y'], inplace = True)

In [41]:
parking_ca.rename(columns = {'spaces_x': 'spaces'}, inplace=True)

In [42]:
print(parking_ca.shape)
print(parking_wa.shape)

(1588, 5)
(281, 5)


In [43]:
ev_parking = pd.concat([parking_ca, parking_wa], ignore_index=True)

In [45]:
ev_parking.isnull().sum()

zip                    0
facility_type         19
ev_slow               35
ev_dc_fast_count      35
spaces              1615
dtype: int64

In [46]:
ev_parking.to_csv('Datasets/us_ev_parking.csv', index = False)